In [ ]:
# --
# Plotly
import plotly as ply
import plotly.graph_objs as go
from plotly.widgets import GraphWidget
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Layout
import plotly.tools as tls
import pandas as pd
# Matplotlib
out_smplot = widgets.Output()
init_notebook_mode(connected=True)

# Plot Y limits
setLimits = False
maxY = 15000
minY = 0

toshow = []
axisshow = []
# meanTable = []

def show_devices_sp(Source):
    _device.options = [s for s in list(records.readings[Source]['devices'].keys())]
    _device.source = Source
    
def show_sensors_sp(Source):
    _sensor_drop.options = [s for s in list(records.readings[_test.value]['devices'][Source]['data'].columns)]
    _sensor_drop.source = Source
    _min_date.value = records.readings[_test.value]['devices'][Source]['data'].index.min()._short_repr
    _max_date.value = records.readings[_test.value]['devices'][Source]['data'].index.max()._short_repr

def clear_all_sp(b):
    with out_smplot:
        clear_output()
        del toshow[:]
        del axisshow[:]
        
def clear_sensor_sp(b):
    with out_smplot:
        clear_output()
        d = [_device.source, _sensor_drop.source, _sensor_drop.value]
        print('Removing', d[0] + ' - ' + d[1] + ' - ' + d[2])
        if d in toshow:
            toshow.remove(d)
        
        
def add_sensor_sp(b):
    dimensions = list()
    with out_smplot:
        clear_output()
        d = [_device.source, _sensor_drop.source, _sensor_drop.value]
        
        if d not in toshow: 
            toshow.append(d)
            
        plot_data = records.readings[toshow[0][0]]['devices'][toshow[0][1]]['data'].loc[:,(toshow[0][2],)]

        if b.slice_time:
            plot_data = plot_data[plot_data.index > _min_date.value]
            plot_data = plot_data[plot_data.index < _max_date.value]
        
        if len(toshow) > 1:
            for i in range(1, len(toshow)):
                plot_data = pd.merge(plot_data, records.readings[toshow[i][0]]['devices'][toshow[i][1]]['data'].loc[:,(toshow[i][2],)], left_index=True, right_index=True)
    
        # Change columns naming
        changed = []
        for i in range(len(plot_data.columns)):
            changed.append(toshow[i][1] + ' - '+ plot_data.columns[i])
        plot_data.columns = changed
        
        subplot_rows = 0
        
        for column in plot_data.columns:
            dimensions.append(dict(label=column, values=plot_data[column]))
            
            print(column, 'added to the plot list')
            # xaxis.append(dict(axisd))
        
        traced = go.Splom(dimensions=dimensions,
                  marker=dict(color='rgb(0,30,230, 0.35)',
                              size=5,
                              #colorscale=pl_colorscaled,
                              line=dict(width=0.5,
                                        color='rgb(230,230,230, 0.30)') ),
                  #text=textd,
                  diagonal=dict(visible=False))


        title = ""
    
        layout = go.Layout(title=title,
                   dragmode='select',
                   width=1000,
                   height=1000,
                   autosize=False,
                   hovermode='closest',
                   plot_bgcolor='rgba(240,240,240, 0.95)')
        global fig
        fig = dict(data=[traced], layout=layout)

def plot_cb_sp(b):
    with out_smplot:
        ply.offline.iplot(fig)
    
def reset_time_sp(b):
    _min_date.value = records.readings[b.src.value].index.min()._short_repr
    _max_date.value = records.readings[b.src.value].index.max()._short_repr

layout=widgets.Layout(width='330px')

# Test dropdown
_test = widgets.Dropdown(options=[k for k in records.readings.keys()], 
                        layout=layout,
                        description = 'Test')

_test_drop = widgets.interactive(show_devices_sp, 
                                Source=_test, 
                                layout=layout)

# Device dropdown
_device = widgets.Dropdown(layout=layout,
                        description = 'Device')

_device_drop = widgets.interactive(show_sensors_sp, 
                                Source=_device, 
                                layout=layout)

# Sensor dropdown
_sensor_drop = widgets.Dropdown(layout=layout,
                               description = 'Channel')

# Buttons
_b_add = widgets.Button(description='Add to Plot', layout=widgets.Layout(width='120px'))
_b_add.on_click(add_sensor_sp)
_b_add.slice_time = False
_b_clear = widgets.Button(description='Clear', layout=widgets.Layout(width='120px'))
_b_clear.on_click(clear_sensor_sp)
_b_reset_all = widgets.Button(description='Clear all', layout=widgets.Layout(width='120px'))
_b_reset_all.on_click(clear_all_sp)
_plot_b = widgets.Button(description='Plot', layout=widgets.Layout(width='120px'))
_plot_b.on_click(plot_cb_sp)

# Date fields
_min_date = widgets.Text(description='Start date:', 
                         layout=widgets.Layout(width='330px'))
_max_date = widgets.Text(description='End date:', 
                         layout=widgets.Layout(width='330px'))

# Date buttons
_b_apply_time = _b_reset = widgets.Button(description='Apply dates', layout=widgets.Layout(width='100px'))
_b_apply_time.on_click(add_sensor_sp)
_b_apply_time.slice_time = True
_b_reset_time = _b_reset = widgets.Button(description='Reset dates', layout=widgets.Layout(width='100px'))
_b_reset_time.on_click(reset_time_sp)

_device_box = widgets.HBox([_test_drop, _device_drop])
_sensor_box = widgets.HBox([_sensor_drop, _b_add, _b_clear, _b_reset_all])

_plot_box = widgets.HBox([_plot_b])
_time_box = widgets.HBox([_min_date,_max_date, _b_reset_time, _b_apply_time])
_root_box = widgets.VBox([_time_box, _device_box, _sensor_box, _plot_box])
display(_root_box)

display(out_smplot)